## Use albumentations library for image augmentation
To generate object detection dataset. Currently works for Pascal VOC format

Notebook author: yptheangel(Choo Wilson)<br>
References: <br>
https://www.curiousily.com/posts/image-data-augmentation-for-tensorflow-2-keras-and-pytorch-with-albumentations-in-python/
https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/example_bboxes.ipynb

In [ ]:
%matplotlib inline
import cv2
import os
import os.path as osp
import xml.etree.ElementTree as ET
import numpy as np
import albumentations as A
from albumentations import (Compose,BboxParams)
from matplotlib import pyplot as plt

In [ ]:
def getObjects(fullname):
    CLS=['avocado','banana']
    bb = ""
    objs=[]
    cls_ids=[]
    in_file = open(fullname)
    tree=ET.parse(in_file)
    root = tree.getroot()
    # iterate through the objects inside a xml file
    for i, obj in enumerate(root.iter('object')):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in CLS or int(difficult)==1:
            continue
        cls_id = CLS.index(cls)
        xmlbox = obj.find('bndbox')
        b = [int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text)]
        c_id = cls_id
        objs.append(b)
        cls_ids.append(c_id)
    return objs,cls_ids

In [ ]:
# Functions to visualize bounding boxes and class labels on an image. 
# Based on https://github.com/facebookresearch/Detectron/blob/master/detectron/utils/vis.py

BOX_COLOR = (0, 200,0)
TEXT_COLOR = (0, 0, 0)

def visualize_bbox(img, bbox, class_id, class_idx_to_name, color=BOX_COLOR, thickness=2):
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color=color, thickness=thickness)
    class_name = class_idx_to_name[class_id]
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (int(x_min), int(y_min - int(1.3 * text_height))), (int(x_min + text_width), int(y_min)), BOX_COLOR, -1)
    cv2.putText(img, class_name, (int(x_min), int(y_min - int(0.3 * text_height))), cv2.FONT_HERSHEY_SIMPLEX, 0.35,TEXT_COLOR, lineType=cv2.LINE_AA)
    return img

def visualize(annotations, category_id_to_name):
    img = annotations['image'].copy()
    for idx, bbox in enumerate(annotations['bboxes']):
        img = visualize_bbox(img, bbox, annotations['category_id'][idx], category_id_to_name)
    plt.figure(figsize=(12, 10))
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
def get_aug(aug, min_area=0., min_visibility=0.):
       return Compose(aug, bbox_params=BboxParams(format='pascal_voc', min_area=min_area, min_visibility=min_visibility, label_fields=['category_id']))

In [ ]:
doc_aug = [
    A.RandomCrop(p=0.5,height=200,width=200),
    A.RandomBrightnessContrast(p=0.6),
    A.Rotate(limit=10, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.2),
]

In [ ]:
category_id_to_name = {0:'avocado',1:'banana'}

In [ ]:
# imageFile = r"C:\Users\choowilson\Desktop\train\avcd0.jpg"
# xmlFile = r"C:\Users\choowilson\Desktop\train\avcd0.xml"
imageFile = r"C:\Users\choowilson\Desktop\train\bnnn1.jpg"
xmlFile = r"C:\Users\choowilson\Desktop\train\bnnn1.xml"

In [ ]:
image = cv2.imread(imageFile)
print(image.shape)

In [ ]:
objects , classIds = getObjects(xmlFile)

In [ ]:
annotations = {'image':image,'bboxes':objects,'category_id':classIds}

In [ ]:
visualize(annotations, category_id_to_name)

In [ ]:
# aug = get_aug([VerticalFlip(p=1)])
aug = get_aug(doc_aug)
augmented = aug(**annotations)
visualize(augmented, category_id_to_name)

In [ ]:
# augmented image is stored in "augmented"
# print(augmented)

In [ ]:
aug_h = augmented['image'].shape[0]
aug_w = augmented['image'].shape[1]

In [ ]:
# By geaxgx
xml_body_1="""<annotation>
        <folder>FOLDER</folder>
        <filename>{FILENAME}</filename>
        <path>{PATH}</path>
        <source>
                <database>Unknown</database>
        </source>
        <size>
                <width>{WIDTH}</width>
                <height>{HEIGHT}</height>
                <depth>3</depth>
        </size>
"""
xml_object=""" <object>
                <name>{CLASS}</name>
                <pose>Unspecified</pose>
                <truncated>0</truncated>
                <difficult>0</difficult>
                <bndbox>
                        <xmin>{XMIN}</xmin>
                        <ymin>{YMIN}</ymin>
                        <xmax>{XMAX}</xmax>
                        <ymax>{YMAX}</ymax>
                </bndbox>
        </object>
"""
xml_body_2="""</annotation>        
"""

def save_to_voc(imgFile, xmlFile, aug, w, h):
    with open(xmlFile,"w") as f:
        f.write(xml_body_1.format(**{'FILENAME':os.path.basename(imgFile), 'PATH':imgFile,'WIDTH':w,'HEIGHT':h}))
        for i,bb in enumerate(aug['bboxes']):            
            f.write(xml_object.format(**{'CLASS':category_id_to_name[aug['category_id'][i]],'XMIN':int(aug['bboxes'][i][0]),'YMIN':int(aug['bboxes'][i][1]),'XMAX':int(aug['bboxes'][i][2]),'YMAX':int(aug['bboxes'][i][3])}))
        f.write(xml_body_2)

In [ ]:
save_to_voc("aug.jpg","aug.xml",augmented,aug_w,aug_h)
cv2.imwrite("aug.jpg",augmented['image'])